In [74]:
# Import pyspark, and connect via findspark
import findspark
findspark.init()
from pyspark import SparkContext 
from pyspark.sql import SQLContext

In [75]:
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [76]:
train = sqlContext.read.csv('train.csv',header = True,inferSchema=True)

In [77]:
train.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

In [78]:
train.limit(10).toPandas()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,2,2008,WD,Normal,208500
1,2,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,5,2007,WD,Normal,181500
2,3,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,9,2008,WD,Normal,223500
3,4,70,RL,60,9550,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,12,2008,WD,Normal,250000
5,6,50,RL,85,14115,Pave,NA,IR1,Lvl,AllPub,...,0,NA,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75,10084,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,8,2007,WD,Normal,307000
7,8,60,RL,NA,10382,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51,6120,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50,7420,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,1,2008,WD,Normal,118000


In [79]:
# Print the dimensions of the data frame. 
print(train.count(), len(train.columns))

1460 81


In [80]:
# because of how many columns there are, we will loop through and get summary stats of 
# every column by groups of 5. 
names = train.columns
for i in range(0, 81, 5):
    train.select(names[i:(i+5)]).describe().show()
# Note the min max of categorical variables is ASCII value.    

+-------+-----------------+------------------+--------+-----------------+------------------+
|summary|               Id|        MSSubClass|MSZoning|      LotFrontage|           LotArea|
+-------+-----------------+------------------+--------+-----------------+------------------+
|  count|             1460|              1460|    1460|             1460|              1460|
|   mean|            730.5|56.897260273972606|    null|70.04995836802665|10516.828082191782|
| stddev|421.6100093688479| 42.30057099381045|    null|24.28475177448321|  9981.26493237915|
|    min|                1|                20| C (all)|              100|              1300|
|    max|             1460|               190|      RM|               NA|            215245|
+-------+-----------------+------------------+--------+-----------------+------------------+

+-------+------+-----+--------+-----------+---------+
|summary|Street|Alley|LotShape|LandContour|Utilities|
+-------+------+-----+--------+-----------+---------+


+-------+------------+------------------+-----------------+----------+----------+
|summary|GarageFinish|        GarageCars|       GarageArea|GarageQual|GarageCond|
+-------+------------+------------------+-----------------+----------+----------+
|  count|        1460|              1460|             1460|      1460|      1460|
|   mean|        null|1.7671232876712328|472.9801369863014|      null|      null|
| stddev|        null|0.7473150101111095|213.8048414533803|      null|      null|
|    min|         Fin|                 0|                0|        Ex|        Ex|
|    max|         Unf|                 4|             1418|        TA|        TA|
+-------+------------+------------------+-----------------+----------+----------+

+-------+----------+------------------+-----------------+------------------+------------------+
|summary|PavedDrive|        WoodDeckSF|      OpenPorchSF|     EnclosedPorch|         3SsnPorch|
+-------+----------+------------------+-----------------+------------

In [81]:
# We can use the distinct function to calculate the number of unique rows. 
# Lets loop through out names list we created above. 

for name in names:
    print(name, "uniqe values :", train.select(name).distinct().count())

Id uniqe values : 1460
MSSubClass uniqe values : 15
MSZoning uniqe values : 5
LotFrontage uniqe values : 111
LotArea uniqe values : 1073
Street uniqe values : 2
Alley uniqe values : 3
LotShape uniqe values : 4
LandContour uniqe values : 4
Utilities uniqe values : 2
LotConfig uniqe values : 5
LandSlope uniqe values : 3
Neighborhood uniqe values : 25
Condition1 uniqe values : 9
Condition2 uniqe values : 8
BldgType uniqe values : 5
HouseStyle uniqe values : 8
OverallQual uniqe values : 10
OverallCond uniqe values : 9
YearBuilt uniqe values : 112
YearRemodAdd uniqe values : 61
RoofStyle uniqe values : 6
RoofMatl uniqe values : 8
Exterior1st uniqe values : 15
Exterior2nd uniqe values : 16
MasVnrType uniqe values : 5
MasVnrArea uniqe values : 328
ExterQual uniqe values : 4
ExterCond uniqe values : 5
Foundation uniqe values : 6
BsmtQual uniqe values : 5
BsmtCond uniqe values : 5
BsmtExposure uniqe values : 5
BsmtFinType1 uniqe values : 7
BsmtFinSF1 uniqe values : 637
BsmtFinType2 uniqe values

In [82]:
# You can perform a crosstab of any of the variables. 
train.crosstab('Heating','HouseStyle').show()

+------------------+------+------+------+------+------+------+------+----+
|Heating_HouseStyle|1.5Fin|1.5Unf|1Story|2.5Fin|2.5Unf|2Story|SFoyer|SLvl|
+------------------+------+------+------+------+------+------+------+----+
|              Wall|     1|     0|     3|     0|     0|     0|     0|   0|
|              OthW|     0|     0|     0|     0|     1|     1|     0|   0|
|             Floor|     0|     0|     1|     0|     0|     0|     0|   0|
|              GasA|   143|    12|   715|     8|     8|   440|    37|  65|
|              Grav|     3|     2|     2|     0|     0|     0|     0|   0|
|              GasW|     7|     0|     5|     0|     2|     4|     0|   0|
+------------------+------+------+------+------+------+------+------+----+



In [83]:
# We can look at the mean housing price of each Heating group

train.groupby('Heating').agg({'SalePrice': 'mean'}).show()

+-------+------------------+
|Heating|    avg(SalePrice)|
+-------+------------------+
|   OthW|          125750.0|
|   Wall|           92100.0|
|  Floor|           72500.0|
|   GasW|166632.16666666666|
|   GasA|182021.19537815126|
|   Grav| 75271.42857142857|
+-------+------------------+



In [84]:
# Lets see how many rows are in each Heating group
train.groupby('Heating').count().show()

+-------+-----+
|Heating|count|
+-------+-----+
|   OthW|    2|
|   Wall|    4|
|  Floor|    1|
|   GasW|   18|
|   GasA| 1428|
|   Grav|    7|
+-------+-----+



In [85]:
# We can also create new columns in the data frame using the withColumns() 
# function
train.withColumn("Sales_X_MoSold", train.SalePrice * train.MoSold) \
            .select("Sales_X_MoSold", "SalePrice", "MoSold").show(5)

train = train.withColumn("Sales_X_MoSold", train.SalePrice * train.MoSold)

+--------------+---------+------+
|Sales_X_MoSold|SalePrice|MoSold|
+--------------+---------+------+
|        417000|   208500|     2|
|        907500|   181500|     5|
|       2011500|   223500|     9|
|        280000|   140000|     2|
|       3000000|   250000|    12|
+--------------+---------+------+
only showing top 5 rows



In [86]:
train.count(), len(train.columns)

(1460, 82)

In [87]:
# save the file as a parquet file. 
train.write.parquet("train.parquet")

In [88]:
# Testing the parquet write we did
ptest = sqlContext.read.parquet("train.parquet")

In [89]:
ptest.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

In [91]:
sc

<SparkContext master=local[*] appName=pyspark-shell>